# Trabajo Práctico 2: Pruebas de validez

Integrantes: 

Luis Paredes 104851

# Enunciado

El objetivo consiste en probar la validez de un programa que calcula $2^{8^{20}}$ módulo
p, utilizando diferentes estrategias y viendo el perfil de costos (tanto en términos de
longitud de la prueba como de tiempo de generación). 

1. Considerar 
    * $a_0 = 2$ 
    * $a_{n+1}=a_n^8$.
2. Considerar 
    * $a_0 = 2$ 
    * $a_{n+1} = a_n^2$
3. Considerar 
    * $a_0 = 2$ 
    * $a_{2n+1}=a_{2n}^2$ 
    * $a_{2n}=a_{2n-1}^4$.
4. Usando 2 columnas.

Tener en cuenta que, dado el grado de las restricciones, se requiere usar un factor de
expansión (blowup) por lo menos tan grande como el grado de las restricciones.

In [1]:
import channel, field, list_utils, merkle, polynomial

import time
from field import FieldElement # Cuerpo Finito
from polynomial import X # Polinomios
from polynomial import interpolate_poly # Interpolacion de Lagrange
from polynomial import prod # Producto de polinomios
from merkle import MerkleTree # Merkle Tree
from channel import Channel # Implements Fiat-Shamir Heuristic

from CP import CP_eval, get_CP # Returns a list of images from the CP
from FRI import FriCommit # Next domain for the FRI operator
from decommitment import decommit_fri # proof algorithm

Para todo el trabajo usaremos
* El primo $p = 3 \cdot 2^{30} + 1 = 3221225473$
* Espacio Finitamente generado $F _{3 \cdot 2^{30} + 1}= F_{3221225473}$

# Afirmacion a probar

Quiero probar que conozco el valor de $2^{8^{20}}$ módulo p

 * $a_0 = 2$ 
 * $a_{n+1}=a_n^8$.

# 1. Genero la traza de ejecucion

Generamos unos cuantos valores de la tabla para tener una idea de como se ve

| a_i         | Valor     | 
|--------------|-----------|
| $a_0$ | 2      | 2        |
| $a_1$ | $2^8$ |
| $a_2$      | $2^{64}$ | 
| $a_3$      | $2^{512}$ |

Observamos que $a_1 = 2^{8^1}$ , $a_2 = 2^{8^2}$, $a_3 = 2^{8^3}$ por ende reescribo la ecuacion como

$$a_{n} = a_o^{8^n} = 2^{8^n}, n>0$$

por ende la tabla se vera asi

| $a_i$         | Valor     | 
|--------------|-----------|
| $a_0$        | $2^{8^{0}}$         |
| $a_1$        | $2^{8^{1}}$ | 
| $a_2$        | $2^{8^{2}}$ |
| $a_3$        | $2^{8^{3}}$ |
| $...$        | $...$ |
|$a_{21}$      |  $2^{8^{20}}$  |

Genero la tabla de ejecucion para los 21 elementos que necesito

In [2]:
print("Generating the trace")

# Tabla de Ejecucion
a = [FieldElement(2)]

# a_{n} = 2^{8^n}
while len(a) < 21:
    a.append(a[0]**(8**(len(a))))

Generating the trace


# 2. Busco un espacio finitamente generado   

Busco elemento generador $g$ tal que el subgrupo sea de orden n y genere el subgrupo 
$G = {g,g^1, g^2, ... , g^n}$, $G[i] = g^i$

Hint: When 𝑘 divides |𝔽×|, $𝑔^𝑘$ generates a group of size |𝔽×| / 𝑘, 
and the n-th power of some FieldElement 𝑥 can be computed by calling x ** n.

---

Queremos que el orden del subgrupo G sea de 32 porque es la potencia de 2 mas pequeña que excede a la traza.

|𝔽×| / 𝑘 = 32

Sabiendo que |𝔽×| = 3221225473

Tomando $k = 100663296$


Tal que 

|𝔽×| / 𝑘 $\approx 32$.

In [3]:
# Nuestro grupo es de orden 32

# Siendo n el orden del subgrupo
# n*k = |𝔽×| , busco k tal que |𝔽×| = 3221225473
n = 32
mod_F = FieldElement.k_modulus
k = mod_F // n

g = FieldElement.generator() ** (k)  # Busco g^k
G = [g ** i for i in range(n)] # Genero el subgrupo de orden 32

# 3. Busco un polinomio asociado a la traza y hago interpolacion extendiendo la traza

Ahora con el polinomio asociado, cada elemento de la traza lo veo como una evaluacion hecha del polinomio $f$ sobre
elementos $g$ (del espacio generador). Esto se llama Reed-Solomon error correction code.

Ahora la traza se ve asi

|$f(g^i)$| $a_i$         | Valor     | 
|----    |--------------|-----------|
|$f(g^0)$ | $a_0$        | $2^{8^{0}}$         |
|$f(g^1)$ | $a_1$        | $2^{8^{1}}$ | 
|$f(g^2)$ | $a_2$        | $2^{8^{2}}$ |
|$f(g^3)$ | $a_3$        | $2^{8^{3}}$ |
| $...$   | $...$        | $...$ |
|$f(g^{21})$ |$a_{21}$      |  $2^{8^{20}}$  |

Evaluamos en un dominio mucho mas grande que G. Tomamos un dominio H que es 8 veces mas grande

 n = 8*32

 |𝔽×| = 3221225473
 
 $k \approx 12582912$

In [4]:
print("We blow up the trace (Extend the trace) 8 times bigger")

n = 8*32
mod_F = FieldElement.k_modulus
k = mod_F // n

w = FieldElement.generator()

h = w ** (k)  # Este es mi generador g^k para el nuevo dominio
H = [h ** i for i in range(n)] # Genero el subgrupo de orden 8*32

eval_domain = [w * x for x in H]

We blow up the trace (Extend the trace) 8 times bigger


# 4. Busco un polinomio asociado y hago interpolacion

In [5]:
f = interpolate_poly(G[:len(a)], a)  ## Este polinomio pasa por todos los puntos que nos interes
f_eval = [f(d) for d in eval_domain] ## Busco la imagen en todos los elementos del dominio extendido

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 21/21 [00:00<00:00, 5938.54it/s]


# 5. Commitment 

Con la traza extendida, ahora creo las hojas del merkle tree a partir de estos 

In [6]:
f_merkle = MerkleTree(f_eval) 

# 6. Channels

Teoricamente el protocolo STARK requiere que dos personas (prover y verifier) interactuen entre si.

En la practica convertimos este protocolo interactivo a uno no interactivo haciendo uso de la heuristica de Fiat-Shamir.

In [7]:
channel = Channel()     # El channel nos dara datos que deberia darnos el verifier
channel.send(f_merkle.root)

El channel va a actuar como nuestro verificador de ahora en adelante. Nos va a proveer con cualquier dato requerido de parte del mismo.

# 7. Generamos CP y FRI layers

Convierto todas las restricciones originales a rational functions.

* $a_0 = 2$

    1) $f(x) = 2$, para $x=g^0$
    
    2) $g^0$ es raiz de $f(x)-2$
    
    3) $$p_o(x) = \frac{f(x) - 2}{x-g^0}$$

In [8]:
## a0 = 2
numer0 = f - 2
denom0 = X - 1
p0 = numer0 / denom0

* $a_{21} = 2^{8^{20}}$
 
    1) $f(x) = 2^{8^{20}}$, para $x=g^{21}$
    
    2) $g^{21}$ es raiz de $f(x)-2^{8^{20}}$
    
    3) $$p_2(x) = \frac{f(x) - 2^{8^{20}}}{x-g^{21}}$$

In [9]:
# a[21] = (a)^{8^20} 
numer1 = f - FieldElement(2)**(8**20)
denom1 = X - g**20
p1 = numer1 / denom1

* $a_{n+1}=a_n^8$

    1) $f(g\cdot x) = f(x)^8$
    
    2) $\{g^i \, , 0 \leq i \leq 20 \}$ es raiz de $f(g\cdot x)- f(x)^8$
    
    3) $$p_1(x) = \frac{f(g\cdot x)-f(x)^8}{\prod_{i=0}^{20}(x-g^i)}$$

In [10]:
# a[n+1] = (an)^8 

numer2 = f(g* X) - f(X)**8
 

lst = [(X - g**i) for i in range(20)]
denom2 = prod(lst)

p2 = numer2 / denom2

Genero el Composition Polynomial como combinacion lineal de estos polinomios.

Los coeficientes del mismo me los provee el channel ya que toma el roll de verificador

In [11]:
cp = get_CP(channel,[p0,p1])
cp_eval = CP_eval(cp,eval_domain)
cp_merkle = MerkleTree(cp_eval)
channel.send(cp_merkle.root)

In [12]:
cp

FRI Folding Operator

In [13]:
fri_polys, fri_domains, fri_layers, fri_merkles = FriCommit(cp, eval_domain, cp_eval, cp_merkle, channel)

# 8. Genero queries y decommitments

In [14]:
len_query = len(eval_domain) - 1
decommit_fri(channel, fri_layers, fri_merkles, f_eval, f_merkle, len_query)

# 9. Vemos la prueba

In [15]:
print(channel.proof)
print(f'Uncompressed proof length in characters: {len(str(channel.proof))}')

['send:8bc5d357865faf1f6cc4f21dcfde6012074ada5b899b0e21893ab4bbbfa3e037', 'receive_random_field_element:2934728597', 'receive_random_field_element:1063968342', 'send:2cc554ed6360bd397754815c199367f538b54dae482c1037d63f355064002f4b', 'receive_random_field_element:939738386', 'send:6e96042c0dab68e08619227a338b3b0965541ac4c2dc9b9a41a29cdb477b215a', 'receive_random_field_element:2516983789', 'send:e47ff6a529fc94d30b16641b1cc54c686e7c4181721d7914cde0e8bc0d615578', 'receive_random_field_element:1591322635', 'send:8a38db01c0fdc00d2fd6c371cf15ac45f2d7484b291192810abed763ff661fa4', 'receive_random_field_element:221227579', 'send:83921e52712932d8a4afb31f5729bdbf2edd99496169008dbc07286b8981002f', 'receive_random_field_element:181381405', 'send:7f2938df004427eaf2307ac625c9886cec0d47276fec5dc1535746f5eec7cc92', 'send:-284756853', 'receive_random_int:130', 'send:-1355327771', "send:['633662b2ff311702bb9263d76ed2373498c06d9b6c69abce3663b7eeba38a356', '41df99704195120edbb05baa1460664d8d7ad0c30fcdd7120

Uncompressed proof length in characters: 19181

# Tiempos

Vuelvo a correr toda la prueba e imprimo en pantalla el tiempo que toma correr cada parte

In [16]:
def modelo_uno():
    
    start = time.time()
    start_all = start
    print("Generating the trace")

    # Tabla de Ejecucion
    a = [FieldElement(2)]

    # a_{n} = 2^{8^n}
    while len(a) < 21:
        a.append(a[0]**(8**(len(a))))

    
    # Siendo n el orden del subgrupo
    # n*k = |𝔽×| , busco k tal que |𝔽×| = 3221225473
    n = 32
    mod_F = FieldElement.k_modulus
    k = mod_F // n

    g = FieldElement.generator() ** (k)  # Busco g^k
    G = [g ** i for i in range(n)] # Genero el subgrupo de orden 32
    

    print("We blow up the trace (Extend the trace) 8 times bigger")
    
    n = 8*32
    mod_F = FieldElement.k_modulus
    k = mod_F // n

    w = FieldElement.generator()

    h = w ** (k)  # Este es mi generador g^k para el nuevo dominio
    H = [h ** i for i in range(n)] # Genero el subgrupo de orden 8*32

    eval_domain = [w * x for x in H]

    f = interpolate_poly(G[:len(a)], a)  ## Este polinomio pasa por todos los puntos que nos interes
    f_eval = [f(d) for d in eval_domain] ## Busco la imagen en todos los elementos del dominio extendido


    f_merkle = MerkleTree(f_eval) 

    channel = Channel()     # El channel nos dara datos que deberia darnos el verifier
    channel.send(f_merkle.root)

    print(f'{time.time() - start}s')
    start = time.time()
    print("Generating the composition polynomial and the FRI layers...")

    ## a0 = 2
    numer0 = f - 2
    denom0 = X - 1
    p0 = numer0 / denom0
    
    # a[21] = (a)^{8^20} 
    numer1 = f - FieldElement(2)**(8**20)
    denom1 = X - g**20
    p1 = numer1 / denom1


    # a[n+1] = (an)^8 
    numer2 = f(g* X) - f(X)**8

    lst = [(X - g**i) for i in range(20)]
    denom2 = prod(lst)
    p2 = numer2 / denom2


    cp = get_CP(channel,[p0,p1,p2])
    cp_eval = CP_eval(cp,eval_domain)
    cp_merkle = MerkleTree(cp_eval)
    channel.send(cp_merkle.root)

    # FRI Folding Operator
    fri_polys, fri_domains, fri_layers, fri_merkles = FriCommit(cp, eval_domain, cp_eval, cp_merkle, channel)

    print(f'{time.time() - start}s')
    start = time.time()
    print("Generating queries and decommitments...")

    len_query = len(eval_domain) - 1
    decommit_fri(channel, fri_layers, fri_merkles, f_eval, f_merkle, len_query)


    # Verificamos
    print(channel.proof)
    print(f'Overall time: {time.time() - start_all}s')
    print(f'Uncompressed proof length in characters: {len(str(channel.proof))}')

    
if __name__ == "__main__":
    modelo_uno()

Generating the trace
We blow up the trace (Extend the trace) 8 times bigger


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 21/21 [00:00<00:00, 6278.00it/s]

0.015474557876586914s
Generating the composition polynomial and the FRI layers...
0.0628349781036377s
Generating queries and decommitments...


['send:8bc5d357865faf1f6cc4f21dcfde6012074ada5b899b0e21893ab4bbbfa3e037', 'receive_random_field_element:2934728597', 'receive_random_field_element:1063968342', 'receive_random_field_element:895926255', 'send:2cc554ed6360bd397754815c199367f538b54dae482c1037d63f355064002f4b', 'receive_random_field_element:1769999173', 'send:4175db994ee38887132240c64ce7665ddef4f7ffc7b097817647fd83eb75b34a', 'receive_random_field_element:1887388507', 'send:9614b3e2b02efaf580287c6f9ab55161f00d041074af3980c6df59c324e8b02a', 'receive_random_field_element:2582971423', 'send:d87d099a70f2fb90cee96c606bc888e410a034647076cf1eb45b912274f7818e', 'receive_random_field_element:2699377672', 'send:36b27317c3d82f9e981578d9253ed0f5eec10cbd8ae6672cdd137ae28cec6531', 'receive_random_field_element:734224929', 'send:29d83976e815ab0ac817b343ff0a655bef029f80eed5299759dfe5dc14cac2d1', 'send:1555725063', 'receive_random_int:227', 'send:-822189231', "send:['633662b2ff311702bb9263d76ed2373498c06d9b6c69abce3663b7eeba38a356', '2e98ef

Vemos que tomamos
- 0.01562 segundos para crear la traza de ejecucion y extender el dominio 8 veces su tamaño
- 0.06302 segundos en generar el Composition Polynomial y hacer FRI layers
- 0.3435 segundos en total

Los tiempos suelen variar ligeramente entre compilaciones

# Modelo 2

Repetimos todo este analisis ahora con los siguientes modelos

2. Considerar 
    * $a_0 = 2$ 
    * $a_{n+1} = a_n^2$
    
Sabieno que $2^{8^{20}} = 2^{{(2^3)}^{20}} = 2^{2^{60}}$

|$f(g^i)$| $a_i$         | Valor     | 
|----    |--------------|-----------|
|$f(g^0)$ | $a_0$        | $2$         |
|$f(g^1)$ | $a_1$        | $2^{2^{1}}$ | 
|$f(g^2)$ | $a_2$        | $2^{2^{2}}$ |
|$f(g^3)$ | $a_3$        | $2^{2^{3}}$ |
| $...$   | $...$        | $...$ |
|$f(g^{60})$ |$a_{60}$   |  $2^{2^{60}}$|

La tabla de ejecucion tendra 61 elementos por lo que se tomara un subgrupo de orden 64.

Los polinomios asociados son

* $a_0 = 2$
 
    1) $f(x) = 2$, para $x=g^0$
    
    2) $g^0$ es raiz de $f(x)-2$
    
    3) $$p_0(x) = \frac{f(x) - 2}{x-g^0}$$
    
* $a_{60} = 2^{2^{60}}$
 
    1) $f(x) = 2^{2^{60}}$, para $x=g^{60}$
    
    2) $g^{60}$ es raiz de $f(x)-2^{2^{60}}$
    
    3) $$p_1(x) = \frac{f(x) - 2^{2^{60}}}{x-g^{60}}$$
 

* $a_{n+1} = a_n^2$

   1) $f(g\cdot x) = f(x)^2$
    
    2) $\{g^i \, , 0 \leq i \leq 59 \}$ es raiz de $f(g\cdot x)- f(x)^2$
    
    3) $$p_2(x) = \frac{f(g\cdot x)-f(x)^2}{\prod_{i=0}^{59}(x-g^i)}$$



In [17]:
def modelo_dos():
    
    start = time.time()
    start_all = start
    print("Generating the trace")

    # Tabla de Ejecucion
    a = [FieldElement(2)]

    # a_{n} = 2^{8^n}
    while len(a) < 61:
        a.append(a[0]**(2**(len(a))))

    
    # Siendo n el orden del subgrupo
    # n*k = |𝔽×| , busco k tal que |𝔽×| = 3221225473
    n = 64
    mod_F = FieldElement.k_modulus
    k = mod_F // n

    g = FieldElement.generator() ** (k)  # Busco g^k
    G = [g ** i for i in range(n)] # Genero el subgrupo de orden 64
    

    print("We blow up the trace (Extend the trace) 8 times bigger")
    
    n = 8*64
    mod_F = FieldElement.k_modulus
    k = mod_F // n

    w = FieldElement.generator()

    h = w ** (k)  # Este es mi generador g^k para el nuevo dominio
    H = [h ** i for i in range(n)] # Genero el subgrupo de orden 8*64

    eval_domain = [w * x for x in H]

    f = interpolate_poly(G[:len(a)], a)  ## Este polinomio pasa por todos los puntos que nos interes
    f_eval = [f(d) for d in eval_domain] ## Busco la imagen en todos los elementos del dominio extendido


    f_merkle = MerkleTree(f_eval) 

    channel = Channel()     # El channel nos dara datos que deberia darnos el verifier
    channel.send(f_merkle.root)

    print(f'{time.time() - start}s')
    start = time.time()
    print("Generating the composition polynomial and the FRI layers...")

    ## a0 = 2
    numer0 = f - 2
    denom0 = X - 1
    p0 = numer0 / denom0
    
    # a[60] = (a)^{8^20} 
    numer1 = f - FieldElement(2)**(2**60)
    denom1 = X - g**60
    p1 = numer1 / denom1


    # a[n+1] = (an)^2
    numer2 = f(g* X) - f(X)**2

    lst = [(X - g**i) for i in range(59)]
    denom2 = prod(lst)
    p2 = numer2 / denom2


    cp = get_CP(channel,[p0,p1,p2])
    cp_eval = CP_eval(cp,eval_domain)
    cp_merkle = MerkleTree(cp_eval)
    channel.send(cp_merkle.root)
    
    print("El polinomio CP es")
    display(cp)
    
    # FRI Folding Operator
    fri_polys, fri_domains, fri_layers, fri_merkles = FriCommit(cp, eval_domain, cp_eval, cp_merkle, channel)

    print(f'{time.time() - start}s')
    start = time.time()
    print("Generating queries and decommitments...")

    len_query = len(eval_domain) - 1
    decommit_fri(channel, fri_layers, fri_merkles, f_eval, f_merkle, len_query)


    # Verificamos
    print(channel.proof)
    print(f'Overall time: {time.time() - start_all}s')
    print(f'Uncompressed proof length in characters: {len(str(channel.proof))}')

    
if __name__ == "__main__":
    modelo_dos()

Generating the trace
We blow up the trace (Extend the trace) 8 times bigger


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 61/61 [00:00<00:00, 2204.26it/s]

0.0792546272277832s
Generating the composition polynomial and the FRI layers...
El polinomio CP es


0.08609700202941895s
Generating queries and decommitments...
['send:53575d36350edfa437b4735412ca12a9d58db85e40913cab8714ed5a79c421f0', 'receive_random_field_element:2728171251', 'receive_random_field_element:1213704555', 'receive_random_field_element:154185519', 'send:2dd16c4b692792fe936bb480323baf93de769dae7100ae3f11ae4d85b76ceb0d', 'receive_random_field_element:2538967403', 'send:dd51d9fc3bb5c690152f53452ed64c46a48bf6969a83ebcc467db09edbd1b23b', 'receive_random_field_element:1478556687', 'send:cacfd76ad954f500d690a7c63dfeb65885a64b84a45330ac740f4ca5a2db0a32', 'receive_random_field_element:2274698976', 'send:8c3e2dff1c944a83d0bf5f4aae8e6614cf946ce0d076782aaee44468c570e197', 'receive_random_field_element:496718302', 'send:fabf3e8fa569ea5caf5faa76cb5e1feded972e0469a4fb52d7139cefb9b549b1', 'receive_random_field_element:2728008242', 'send:df0231bdb241392c0645fb1aaec43c52409e13fce13e427e5bc9d6337ad4459b', 'receive_random_field_element:1192744081', 'send:6b71c8a455e3a9e4421405fa69aa3bd61c9b

Obtenermos al final Uncompressed proof length in characters: 23796

Vemos que tomamos
- 0.075078 segundos para crear la traza de ejecucion y extender el dominio 8 veces su tamaño
- 0.08616 segundos en generar el Composition Polynomial y hacer FRI layers
- 0.4645 segundos en total

Observamos que tiene un desempeño peor que el modelo anterior. Esto se debe a que al ser la tabla de ejecucion mas larga entonces el problema es mas largo en general ya que el polinomio asociado va a ser de grado mayor.

# Modelo 3

2. Considerar 
    * $a_0 = 2$ 
    * $a_{2n+1}=a_{2n}^2$ 
    * $a_{2n}=a_{2n-1}^4$.
    
Sabieno que $2^{8^{20}}$

|$f(g^i)$| $a_i$         | Valor     
| :----:    | :--------------: |:------------
|$f(g^0)$ | $a_0$        | $2$ |
|$f(g^1)$ | $a_1$        | $2^{2}$ | 
|$f(g^2)$ | $a_2$        | $2^{8}$ |
|$f(g^3)$ | $a_3$        | $2^{8*2}$ |
|$f(g^4)$ | $a_4$        | $2^{8^2}$ |
|$f(g^5)$ | $a_5$        | $2^{8^2*2}$ |
|$f(g^6)$ | $a_6$        | $2^{8^{3}}$ |
|$f(g^7)$ | $a_7$        | $2^{8^{3}*2}$ |
|$f(g^8)$ | $a_8$        | $2^{8^{4}}$ |
| $...$   | $...$        | $...$ |
|$f(g^{40})$ |$a_{40}$   |  $2^{2^{8^{20}}}$|

Observo el patron 

* $a_{2n+1}=2^{8^n \cdot 2}$

* $a_{2n}=2^{8^n}$

De ahi es inmediato que $2^{8^n} = 2^{8^{20}} \rightarrow n=40$ Por ende $a_{40} = 2^{8^{20}}$


La tabla de ejecucion tendra 41 elementos por lo que se tomara un subgrupo de orden 64.

Los polinomios asociados son

* $a_0 = 2$

    1) $f(x) = 2$, para $x=g^0$

    2) $g^0$ es raiz de $f(x)-2$

    3) $$p_0(x) = \frac{f(x) - 2}{x-g^0}$$
 
* $a_{40} = 2^{8^{20}}$
 
    1) $f(x) = 2^{8^{20}}$, para $x=g^{40}$
    
    2) $g^{40}$ es raiz de $f(x)-2^{8^{20}}$
    
    3) $$p_1(x) = \frac{f(x) - 2^{8^{20}}}{x-g^{40}}$$

* $a_{2n+1}=a_{2n}^2$ 

    1) $f(x \cdot g) = f(x)^2, x \in \{g^0,g^2,g^4, ..., g^{38} \}$
    
    2) $\{ g^{2 \cdot i} , 0 \leq i \leq 19 \}$ es raiz de $f(x \cdot g) - f(x)^2$
    
    3) $$p_2(x) = \frac{f(x \cdot g) - f(x)^2}{{\prod_{i=0}^{19}(x-g^{2i})}}$$

* $a_{2n}=a_{2n-1}^4$

    1) $f(x \cdot g) = f(x)^4, x \in \{g^1,g^3,g^5, ..., g^{39} \}$
    
    2) $\{ g^{(2 i + 1)} , 0 \leq i \leq 19 \}$ es raiz de $f(x \cdot g) - f(x)^4$
    
    3) $$p_3(x) = \frac{f(x \cdot g) - f(x)^4}{{\prod_{i=0}^{19}(x-g^{(2 i + 1) })}}$$

In [18]:
def modelo_tres():
    
    start = time.time()
    start_all = start
    print("Generating the trace")

   # Tabla de Ejecucion
    a = [FieldElement(2)] # a0 = 2

    # a_{n} = 2^{8^n}
    while len(a) < 41:
        a.append(a[-1]**2) # 𝑎_{2𝑛+1}=𝑎_{2𝑛}^2
        a.append(a[-1]**4) # 𝑎_{2𝑛}=𝑎_{2𝑛−1}^4
    
    # Siendo n el orden del subgrupo
    # n*k = |𝔽×| , busco k tal que |𝔽×| = 3221225473
    n = 64
    mod_F = FieldElement.k_modulus
    k = mod_F // n

    g = FieldElement.generator() ** (k)  # Busco g^k
    G = [g ** i for i in range(n)] # Genero el subgrupo de orden 64
    

    print("We blow up the trace (Extend the trace) 8 times bigger")
    
    n = 8*64
    mod_F = FieldElement.k_modulus
    k = mod_F // n

    w = FieldElement.generator()

    h = w ** (k)  # Este es mi generador g^k para el nuevo dominio
    H = [h ** i for i in range(n)] # Genero el subgrupo de orden 8*64

    eval_domain = [w * x for x in H]

    f = interpolate_poly(G[:len(a)], a)  ## Este polinomio pasa por todos los puntos que nos interes
    f_eval = [f(d) for d in eval_domain] ## Busco la imagen en todos los elementos del dominio extendido


    f_merkle = MerkleTree(f_eval) 

    channel = Channel()     # El channel nos dara datos que deberia darnos el verifier
    channel.send(f_merkle.root)

    print(f'{time.time() - start}s')
    start = time.time()
    print("Generating the composition polynomial and the FRI layers...")

    ## a0 = 2
    numer0 = f - 2
    denom0 = X - 1
    p0 = numer0 / denom0

    
    # a[40] = (a)^{8^20} 
    numer1 = f - FieldElement(2)**(8**20)
    denom1 = X - g**40
    p1 = numer1 / denom1
    
    # 𝑎[2𝑛+1]=𝑎[2𝑛]^2
    numer2 = f(g* X) - f(X)**2

    lst = [(X - g**(2*i)) for i in range(0,19)]
    denom2 = prod(lst)
    p2 = numer2 / denom2
    
    # 𝑎[2𝑛+1]=𝑎[2𝑛]^2
    numer3 = f(g* X) - f(X)**4

    lst = [(X - g**(2*i+1)) for i in range(0,19)]
    denom3 = prod(lst)
    p3 = numer3 / denom3
    

    cp = get_CP(channel,[p0,p1,p2,p3])
    cp_eval = CP_eval(cp,eval_domain)
    cp_merkle = MerkleTree(cp_eval)
    channel.send(cp_merkle.root)
    
    
    print("El polinomio CP es")
    display(cp)

    # FRI Folding Operator
    fri_polys, fri_domains, fri_layers, fri_merkles = FriCommit(cp, eval_domain, cp_eval, cp_merkle, channel)

    print(f'{time.time() - start}s')
    start = time.time()
    print("Generating queries and decommitments...")

    len_query = len(eval_domain) - 1
    decommit_fri(channel, fri_layers, fri_merkles, f_eval, f_merkle, len_query)


    # Verificamos
    print(channel.proof)
    print(f'Overall time: {time.time() - start_all}s')
    print(f'Uncompressed proof length in characters: {len(str(channel.proof))}')

    
if __name__ == "__main__":
    modelo_tres()

Generating the trace
We blow up the trace (Extend the trace) 8 times bigger


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 41/41 [00:00<00:00, 3456.13it/s]

0.031846046447753906s
Generating the composition polynomial and the FRI layers...
El polinomio CP es


0.08926200866699219s
Generating queries and decommitments...
['send:8990711248b43781f8fffab1507823615670122853e65d59a29967fa73937d80', 'receive_random_field_element:764246289', 'receive_random_field_element:2837830382', 'receive_random_field_element:2201253843', 'receive_random_field_element:2649127945', 'send:f563ce73fffd8ca45a4b1a9e3cdb5dddb5f4184fce36836f4892a4d48cffed96', 'receive_random_field_element:2288949843', 'send:4284eaa2e1a17ee4ca411490296ad02ae56081b8b8c8a945935e7ccc7ca3d8a4', 'receive_random_field_element:423632064', 'send:8f2c5f2fbcde3b2cda26de1194126db9c4b1c96f75fa7ebbb6ff944910ce8f94', 'receive_random_field_element:2753483072', 'send:261392f201480d09bfd05112ea39ca060eb9f47525d7d8bb61cde6a5d57f7ba0', 'receive_random_field_element:395691077', 'send:bab70f37bba1af271c6003f6e5831b4d9b8ed12fb4ae79b4a972600b5b9da679', 'receive_random_field_element:742971178', 'send:ce75b155cd6c66deeff6070328a02f18f8ddb21dfbf1287193eeca72a07bd177', 'receive_random_field_element:1064546337', '

Obtenermos al final Uncompressed proof length in characters: 23828

Vemos que tomamos
- 0.03218 segundos para crear la traza de ejecucion y extender el dominio 8 veces su tamaño
- 0.088959 segundos en generar el Composition Polynomial y hacer FRI layers
- 0.436426 segundos en total

Observamos que tiene un desempeño peor que el primer modelo pero mejor que el segundo. 

Se ve reflejado como la diferencia en la cantidad de elementos en la traza de ejecucion afecta el tiempo computacional.

# Modelo 4 (To-Do)

Usando 2 columnas.

# Analisis de Datos

En general la diferencia entre modelos es muy pequeña. 

Sin embargo, a gran escala el margen se va haciendo mayor asi que al analizar los modelos probados el que mejor desempeño tiene es el primero ya que genera menos valores en la tabla de traza.

Tambien se puede apreciar como mientras mas extensa se hace la traza mas demanda computacional (tiempo de computo) exige cada modelo. 

# Conclusion

El protocolo STARK es una protocolo que nos permite demostrar o validar el trabajo hecho sin tener que hacerlo de vuelta o entregar datos personales que demuestren lo que afirmamos.

Con las pruebas de validez podemos probar que un cómputo se ha realizado correctamente en un tiempo bastante menor que la verificación trivial, es decir, que el cómputo sea corrido de manera independiente por el verificador. 

Ignorando un momento toda la matematica de trasfondo, el protocolo actua casi como magia y el hecho de que no requiera de interaccion externa le da una seguridad enorme. 

Sin embargo, con estas soluciones tambien creamos nuevos problemas. Como dependemos fuertemente del channel es critico que su aleatoridad sea absoluta y en el mundo real esto es algo que no podemos afirmar a ciegas. Si algun dia se descubre una manera de predecir los valores que nos da entonces todo el protocolo se romperia. 

Pero como todo en criptografia, el protocolo funciona bien hasta que inevitablemente alguien lo rompa. Por el momento, STARK parece ser muy prometedor para el futuro de la criptografia y ya hay planes para incorporarlo como una mejora de su protocolo hermano SNARK.